# Transition-based arc-eager unlabeled dependency parser for Ukrainian

## Read the data

Useful links:
* [UD corpus for Ukrainian](https://github.com/UniversalDependencies/UD_Ukrainian-IU/)
* [Easy-to-use library for parsing UD](https://github.com/EmilStenstrom/conllu)

In [32]:
from collections import OrderedDict
from conllu import parse
from enum import Enum

PATH = "UD_Ukrainian-IU"

with open(PATH + "/uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

trees = parse(data)

In [33]:
tree = trees[0]
print(tree[0])

OrderedDict([('id', 1), ('form', 'У'), ('lemma', 'у'), ('upostag', 'ADP'), ('xpostag', 'Spsl'), ('feats', OrderedDict([('Case', 'Loc')])), ('head', 2), ('deprel', 'case'), ('deps', [('case', 2)]), ('misc', OrderedDict([('Id', '0003'), ('LTranslit', 'u'), ('Translit', 'U')]))])


In [34]:
for node in tree:
    head = node["head"]
    print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


## Design actions and the oracle

We will be using a static oracle that reproduces a single valid order of actions.

In [35]:
class Actions(str, Enum):
    SHIFT = "shift"
    REDUCE = "reduce"
    RIGHT = "right"
    LEFT = "left"

In [36]:
def oracle(stack, top_queue, relations):
    """
    Make a decision on the right action to do.
    """
    top_stack = stack[-1]
    # check if both stack and queue are non-empty
    if top_stack and not top_queue:
        return Actions.REDUCE
    # check if there are any clear dependencies
    elif top_queue["head"] == top_stack["id"]:
        return Actions.RIGHT
    elif top_stack["head"] == top_queue["id"]:
        return Actions.LEFT
    # check if we can reduce the top of the stack
    elif top_stack["id"] in [i[0] for i in relations] and \
         (top_queue["head"] < top_stack["id"] or \
          [s for s in stack if s["head"] == top_queue["id"]]):
        return Actions.REDUCE
    # default option
    else:
        return Actions.SHIFT

In [37]:
ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                    ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                    ('deps', None), ('misc', None)])

def trace_actions(tree, log=True):
    """
    Try out the oracle to verify it's returning the right actions.
    """
    stack, queue, relations = [ROOT], tree[:], []
    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        if log:
            print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            print("Relations:", relations)
            print(action)
            print("========================")
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    if log:
        print("Gold relations:")
        print([(node["id"], node["head"]) for node in tree])
        print("Retrieved relations:")
        print(sorted(relations))

trace_actions(tree)

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.LEFT
Stack: ['ROOT_0']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'В

## Feature extraction

Reference: [Dependency Parsing by Kübler, McDonald, and Nivre](https://books.google.com.ua/books?id=k3iiup7HB9UC&pg=PA21&hl=uk&source=gbs_toc_r&cad=4#v=onepage&q&f=false)

In [38]:
def extract_features(stack, queue):
    features = dict()
    if len(stack) > 0:
        stack_top = stack[-1]
        features["s0-word"] = stack_top["form"]
        features["s0-lemma"] = stack_top["lemma"]
        features["s0-tag"] = stack_top["upostag"]
        if stack_top["feats"]:
            for k, v in stack_top["feats"].items():
                features["s0-" + k] = v
    if len(stack) > 1:
        features["s1-tag"] = stack_top["upostag"]
    if queue:
        queue_top = queue[0]
        features["q0-word"] = queue_top["form"]
        features["q0-lemma"] = queue_top["lemma"]
        features["q0-tag"] = queue_top["upostag"]
        if queue_top["feats"]:
            for k, v in queue_top["feats"].items():
                features["q0-" + k] = v
    if len(queue) > 1:
        queue_next = queue[1]
        features["q1-word"] = queue_next["form"]
        features["q1-tag"] = queue_next["upostag"]
    if len(queue) > 2:
        features["q2-tag"] = queue[2]["upostag"]
    if len(queue) > 3:
        features["q3-tag"] = queue[3]["upostag"]
    if stack and queue:
        features["distance"] = queue[0]["id"] - stack[-1]["id"]
    return features

## Prepare train and test data

In [39]:
def get_data(tree):
    features, labels = [], []
    stack, queue, relations = [ROOT], tree[:], []

    while queue or stack:
        action = oracle(stack if len(stack) > 0 else None,
                        queue[0] if len(queue) > 0 else None,
                        relations)
        features.append(extract_features(stack, queue))
        labels.append(action.value)
        if action == Actions.SHIFT:
            stack.append(queue.pop(0))
        elif action == Actions.REDUCE:
            stack.pop()
        elif action == Actions.LEFT:
            relations.append((stack[-1]["id"], queue[0]["id"]))
            stack.pop()
        elif action == Actions.RIGHT:
            relations.append((queue[0]["id"], stack[-1]["id"]))
            stack.append(queue.pop(0))
        else:
            print("Unknown action.")
    return features, labels

In [40]:
features, labels = get_data(tree)
print("Number of words:", len(tree))
print("Number of actions:", len(labels))
print("List of actions taken:", labels)
print("Features:")
for word in features:
    print(word)

Number of words: 14
Number of actions: 29
List of actions taken: ['shift', 'left', 'shift', 'shift', 'left', 'right', 'right', 'reduce', 'reduce', 'left', 'right', 'shift', 'left', 'right', 'shift', 'left', 'right', 'right', 'shift', 'left', 'right', 'reduce', 'reduce', 'reduce', 'reduce', 'right', 'reduce', 'reduce', 'reduce']
Features:
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'У', 'q0-lemma': 'у', 'q0-tag': 'ADP', 'q0-Case': 'Loc', 'q1-word': 'домі', 'q1-tag': 'NOUN', 'q2-tag': 'ADJ', 'q3-tag': 'NOUN', 'distance': 1}
{'s0-word': 'У', 's0-lemma': 'у', 's0-tag': 'ADP', 's0-Case': 'Loc', 's1-tag': 'ADP', 'q0-word': 'домі', 'q0-lemma': 'дім', 'q0-tag': 'NOUN', 'q0-Animacy': 'Inan', 'q0-Case': 'Loc', 'q0-Gender': 'Masc', 'q0-Number': 'Sing', 'q1-word': 'римського', 'q1-tag': 'ADJ', 'q2-tag': 'NOUN', 'q3-tag': 'PROPN', 'distance': 1}
{'s0-word': 'ROOT', 's0-lemma': 'ROOT', 's0-tag': 'ROOT', 'q0-word': 'домі', 'q0-lemma': 'дім', 'q0-tag': 'NOUN', 'q0-Animacy': 'I

In [41]:
train_features, train_labels = [], []
for tree in trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    train_features += tree_features
    train_labels += tree_labels

print(len(train_features), len(train_labels))

190298 190298


In [42]:
# Test data

with open(PATH + "/uk_iu-ud-test.conllu", "r") as f:
    data = f.read()
test_trees = parse(data)

test_features, test_labels = [], []
for tree in test_trees:
    tree_features, tree_labels = get_data([t for t in tree if type(t["id"])==int])
    test_features += tree_features
    test_labels += tree_labels

print(len(test_features), len(test_labels))

35124 35124


## Train a classifier

In [57]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
import numpy as np

In [47]:
vectorizer = DictVectorizer()
vec = vectorizer.fit(train_features)

print("\nTotal number of features: ", len(vec.get_feature_names()))


Total number of features:  111260


In [48]:
train_features_vectorized = vec.transform(train_features)
test_features_vectorized = vec.transform(test_features)

print(len(train_features_vectorized.toarray()), len(test_features_vectorized.toarray()))

190298 35124


In [53]:
mnb = MultinomialNB()
mnb_pred = mnb.fit(train_features_vectorized, train_labels)

In [49]:
lrc = LogisticRegression(random_state=42, solver="sag", multi_class="multinomial", max_iter=1000, verbose=1)
lrc.fit(train_features_vectorized, train_labels)

max_iter reached after 361 seconds


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.0min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=1, warm_start=False)

In [69]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 15), random_state=3)
clf.fit(train_features_vectorized, train_labels)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 15), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=3, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [84]:
predicted_clf = clf.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_clf))

             precision    recall  f1-score   support

       left       0.84      0.89      0.87      8658
     reduce       0.81      0.72      0.76      9350
      right       0.75      0.80      0.78      8291
      shift       0.90      0.90      0.90      8825

avg / total       0.83      0.83      0.82     35124



In [85]:
predicted_lrc = lrc.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_lrc))

             precision    recall  f1-score   support

       left       0.85      0.87      0.86      8658
     reduce       0.82      0.72      0.77      9350
      right       0.74      0.80      0.77      8291
      shift       0.87      0.89      0.88      8825

avg / total       0.82      0.82      0.82     35124



In [86]:
predicted_mnb = mnb.predict(test_features_vectorized)
print(classification_report(test_labels, predicted_mnb))

             precision    recall  f1-score   support

       left       0.76      0.86      0.81      8658
     reduce       0.72      0.47      0.57      9350
      right       0.63      0.77      0.69      8291
      shift       0.77      0.78      0.78      8825

avg / total       0.72      0.72      0.71     35124



In [71]:
from sklearn.externals import joblib
joblib.dump(clf, 'MLPClassifier.sav')

['MLPClassifier.sav']

## Calculate the unlabeled attachment score
UAS - the percentage of words in an input that are assigned the correct head.

In [94]:
def dep_parse(sentence, oracle, vectorizer, log=True):
    stack, queue, relations = [ROOT], sentence[:], []
    while queue or stack:
        if stack and not queue:
            stack.pop()
        else:
            features = extract_features(stack, queue)
            action = oracle.predict(vectorizer.transform([features]))[0]
            #if log:
            #   print("Stack:", [i["form"]+"_"+str(i["id"]) for i in stack])
            #    print("Queue:", [i["form"]+"_"+str(i["id"]) for i in queue])
            #    print("Relations:", relations)
            #    print(action)
            #    print("========================")
            # actual parsing
            if action == Actions.SHIFT:
                stack.append(queue.pop(0))
            elif action == Actions.REDUCE:
                stack.pop()
            elif action == Actions.LEFT:
                relations.append((stack[-1]["id"], queue[0]["id"]))
                stack.pop()
            elif action == Actions.RIGHT:
                relations.append((queue[0]["id"], stack[-1]["id"]))
                stack.append(queue.pop(0))
            else:
                print("Unknown action.")
    return sorted(relations)

In [101]:
def print_results(model):
    total, tp = 0, 0
    for tree in test_trees:
        tree = [t for t in tree if type(t["id"])==int]
        golden = [(node["id"], node["head"]) for node in tree]
        predicted = dep_parse(tree, model, vec)
        total += len(tree)
        tp += len(set(golden).intersection(set(predicted)))

    print("Total:", total)
    print("Correctly defined:", tp)
    print("UAS:", round(tp/total, 2))

In [103]:
print_results(clf)

Total: 17116
Correctly defined: 12164
UAS: 0.71


In [90]:
print_results(lrc)

Total: 17116
Correctly defined: 12149
UAS: 0.71
